<a href="https://colab.research.google.com/github/Matthieu-Romain/ToyStore-ecommerce/blob/main/Brief2_Friends.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Brief 2

On considère le site: www.imdb.com

**1- Analyse du Site Web :**

On considère le site imdb et cette URL : https://www.imdb.com/title/tt0108778/episodes/ .

inspectez la structure HTML pour comprendre comment les informations de la série ,ses épisodes et les différentes saisons sont présentées.

Identifiez les balises HTML, les classes et les IDs pertinents qui contiennent les données que vous souhaitez extraire.

**2-Extraction des Informations de Base :**

* 2.1- les épisodes de la saison 1:

Utilisez Beautiful Soup pour envoyer une requête HTTP à la page https://www.imdb.com/title/tt0108778/episodes/  récupérez le contenu HTML.

Récupérez la liste des episodes de la 1ere saison.

Récupérer pour chaque épisode , son numéro, son nom, la date de sortie, la note et la description

* 2.2-Récupérer toutes les saisons  

Etendre votre travail précédent pour extraire les les épisodes des 10 saisons de la série



In [71]:
import requests
from bs4 import BeautifulSoup

In [72]:


# L'url du site à scrapper
url = 'https://www.imdb.com/title/tt0108778/episodes/'

# ma requete HTTP avec un "GET" au serveur du site identifier dans l'url
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)

# J'affiche l'url ainsi que le retour du serveur
print(url, response.status_code)



https://www.imdb.com/title/tt0108778/episodes/ 200


In [73]:
# Je demande à beautifulSoup de conserver dans une variable "soup" la page web à scrapper (un script html)
soup = BeautifulSoup(response.content,'html')
#soup

In [74]:
soup.select_one('h2').text

'六人行'

In [75]:
soup.select_one('.ipc-title__text.ipc-title__text--reduced').text


'Episode list'

In [76]:
title_tag = soup.select_one('.ipc-title__text.ipc-title__text--reduced')
print(title_tag.text.strip() if title_tag else "Title not found")

Episode list


In [77]:
print(soup.prettify()[:1000])  # Preview  du code html pour s assurer qu il est bien là, bizarement on a du contenu en chinois

<!DOCTYPE html>
<html lang="en-US" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <script>
   window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1754643820463);
        }
    })
  </script>
  <title>
   六人行 (TV Series 1994–2004) - Episode list - IMDb
  </title>
  <meta content="六人行 (TV Series 1994–2004) - Movies, TV, Celebs, and more..." data-id="main" name="description"/>
  <meta content="Episodes" name="keywords"/>
  <meta con

In [78]:
print(response.status_code)  # Should be 200
print(soup.title.text)       # Should mention "Friends"


200
六人行 (TV Series 1994–2004) - Episode list - IMDb


In [66]:
import re
import pandas as pd


In [87]:
#Creation d une liste des épisodes itération d une boucle , recupération des titres avec une regular expression,
episodes = []
for a in soup.select("article.episode-item-wrapper"):

    titre_brut = a.select_one("div.ipc-title__text.ipc-title__text--reduced").text.strip()
    match = re.search(r"(S\d+)\.(\s*E\d+)\s*∙\s*(.*)", titre_brut)
    saison = match.group(1).strip()
    numero = match.group(2).strip()
    titre = match.group(3).strip()

    description = a.select_one("div.ipc-html-content-inner-div").text.strip()

    date = a.select_one("div.sc-5372d523-5.jBUStp span").text.strip()

    note = a.select_one("span.ipc-rating-star--rating").text.strip()

    nbre_votants = a.select_one("span.ipc-rating-star--voteCount").text.strip()
#génération d'un dataframe
    episodes.append({'titre': titre, "saison": saison, "épisode": numero, "résumé": description, "date de diffusion": date, 'note': note, "nombre de votants": nbre_votants})
df_saison1 = pd.DataFrame(episodes)
df_saison1.head(10)
# Tri par note décroissante
#df_saison1_sorted = df_saison1.sort_values(by="note", ascending=False)

# Affichage des 10 meilleurs épisodes
#print(df_saison1_sorted.head(10))

,titre,saison,épisode,résumé,date de diffusion,note,nombre de votants
0,The One Where Monica Gets a Roommate,S1,E1,"Monica and the gang introduce Rachel to the ""r...","Sun, Dec 31, 2017",8.1,(11K)
1,The One with the Sonogram at the End,S1,E2,Ross finds out his ex-wife is pregnant. Rachel...,"Thu, Sep 29, 1994",7.8,(8.6K)
2,The One with the Thumb,S1,E3,Monica becomes irritated when everyone likes h...,"Thu, Oct 6, 1994",8.0,(8.1K)
3,The One with George Stephanopoulos,S1,E4,Joey and Chandler take Ross to a hockey game t...,"Thu, Oct 13, 1994",7.9,(7.9K)
4,The One with the East German Laundry Detergent,S1,E5,"Eager to spend time with Rachel, Ross pretends...","Thu, Oct 20, 1994",8.3,(7.9K)
5,The One with the Butt,S1,E6,Monica's obsessiveness is put to the test afte...,"Thu, Oct 27, 1994",8.0,(7.5K)
6,The One with the Blackout,S1,E7,"When New York suffers from a blackout, Ross tr...","Thu, Nov 3, 1994",8.8,(9.3K)
7,The One Where Nana Dies Twice,S1,E8,Monica and Ross mourn the loss of their grandm...,"Thu, Nov 10, 1994",7.9,(7.3K)
8,The One Where Underdog Gets Away,S1,E9,The gang's plans for Thanksgiving go awry afte...,"Thu, Nov 17, 1994",8.1,(7.1K)
9,The One with the Monkey,S1,E10,The gang make (and break) a pact not to bring ...,"Thu, Dec 15, 1994",7.9,(7.1K)


In [88]:
# Tri par note décroissante
df_saison1_sorted = df_saison1.sort_values(by="note", ascending=False)

# Affichage des 10 meilleurs épisodes de la saison 1
print(df_saison1_sorted.head(10))

                                             titre saison épisode  \
6                        The One with the Blackout     S1      E7   
23                  The One Where Rachel Finds Out     S1     E24   
17                      The One with All the Poker     S1     E18   
22                          The One with the Birth     S1     E23   
12                        The One with the Boobies     S1     E13   
16                  The One with Two Parts: Part 2     S1     E17   
4   The One with the East German Laundry Detergent     S1      E5   
14                     The One with the Stoned Guy     S1     E15   
13                   The One with the Candy Hearts     S1     E14   
21                     The One with the Ick Factor     S1     E22   

                                               résumé  date de diffusion note  \
6   When New York suffers from a blackout, Ross tr...   Thu, Nov 3, 1994  8.8   
23  As Ross leaves for China, Chandler accidentall...  Thu, May 18, 1995  8.7 

In [68]:
print(episodes)

[{'titre': 'The One Where Monica Gets a Roommate', 'saison': 'S1', 'épisode': 'E1', 'résumé': 'Monica and the gang introduce Rachel to the "real world" after she leaves her fiancé at the altar.', 'date de diffusion': 'Sun, Dec 31, 2017', 'note': '8.1', 'nombre de votants': '(11K)'}, {'titre': 'The One with the Sonogram at the End', 'saison': 'S1', 'épisode': 'E2', 'résumé': "Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross's parents come to visit.", 'date de diffusion': 'Thu, Sep 29, 1994', 'note': '7.8', 'nombre de votants': '(8.6K)'}, {'titre': 'The One with the Thumb', 'saison': 'S1', 'épisode': 'E3', 'résumé': 'Monica becomes irritated when everyone likes her new boyfriend more than she does. Chandler resumes his smoking habit. Phoebe is given $7000 when she finds a thumb in a can of soda.', 'date de diffusion': 'Thu, Oct 6, 1994', 'note': '8.0', 'nombre de votants': '(8.1K)'}, {'titre': 'The One with Geo

In [97]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.imdb.com/title/tt0108778/episodes/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

episodes2 = soup.select('article.episode-item-wrapper')

for ep in episodes2:
    title_tag = ep.select_one("div.ipc-title__text.ipc-title__text--reduced")
    description_tag = ep.select_one("div.ipc-html-content-inner-div")
    airdate_tag = ep.select_one("div.sc-5372d523-5.jBUStp span")
    rating_tag = ep.select_one("span.ipc-rating-star--rating")

    print("Titre :", title_tag.text.strip() if title_tag else "N/A")
    print("Description :", description_tag.text.strip() if description_tag else "N/A")
    print("Date de sortie :", airdate_tag.text.strip() if airdate_tag else "N/A")
    print("Note :", rating_tag.text.strip() if rating_tag else "N/A")
    print("-" * 50)

Titre : S1.E1 ∙ The One Where Monica Gets a Roommate
Description : Monica and the gang introduce Rachel to the "real world" after she leaves her fiancé at the altar.
Date de sortie : Sun, Dec 31, 2017
Note : 8.1
--------------------------------------------------
Titre : S1.E2 ∙ The One with the Sonogram at the End
Description : Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross's parents come to visit.
Date de sortie : Thu, Sep 29, 1994
Note : 7.8
--------------------------------------------------
Titre : S1.E3 ∙ The One with the Thumb
Description : Monica becomes irritated when everyone likes her new boyfriend more than she does. Chandler resumes his smoking habit. Phoebe is given $7000 when she finds a thumb in a can of soda.
Date de sortie : Thu, Oct 6, 1994
Note : 8.0
--------------------------------------------------
Titre : S1.E4 ∙ The One with George Stephanopoulos
Description : Joey and Chandler take Ro

In [70]:
print(episodes2[0].prettify())

<article class="sc-64257d69-1 mOJzu episode-item-wrapper">
 <div class="ipc-list-card--base ipc-list-card sc-5372d523-0 bLOGkO">
  <div class="sc-5372d523-1 bHnYuv">
   <div aria-label="Add to Watchlist" class="ipc-watchlist-ribbon ipc-focusable ipc-watchlist-ribbon--m ipc-watchlist-ribbon--base ipc-watchlist-ribbon--onImage sc-5372d523-3 iaTaBB" data-testid="watchlist-ribbon-add" role="button" tabindex="0">
    <svg class="ipc-watchlist-ribbon__bg" height="34px" role="presentation" viewbox="0 0 24 34" width="24px" xmlns="http://www.w3.org/2000/svg">
     <polygon class="ipc-watchlist-ribbon__bg-ribbon" fill="#000000" points="24 0 0 0 0 32 12.2436611 26.2926049 24 31.7728343">
     </polygon>
     <polygon class="ipc-watchlist-ribbon__bg-hover" points="24 0 0 0 0 32 12.2436611 26.2926049 24 31.7728343">
     </polygon>
     <polygon class="ipc-watchlist-ribbon__bg-shadow" points="24 31.7728343 24 33.7728343 12.2436611 28.2926049 0 34 0 32 12.2436611 26.2926049">
     </polygon>
    </s

In [120]:
#Récupération des épisodes de toutes les saisons
all_episodes = []

# iterer une boucle des saisons Friends 1 à 10 avec season_num comme itérable
for season_num in range(1, 11):
    url = f"https://www.imdb.com/title/tt0108778/episodes?season={season_num}"
    #url = "https://www.imdb.com/title/tt0108778/episodes?season={}"
    #url = "https://www.imdb.com/title/tt0108778/episodes?season={}".format(season_num)


    response = requests.get(url, headers=headers)
#Faire le parsing du html en déclarant une variable soup
    soup = BeautifulSoup(response.content, "html.parser")
#Extraire les données du titre avec une regex
    for a in soup.select("article.episode-item-wrapper"):
        titre_brut = a.select_one("div.ipc-title__text.ipc-title__text--reduced").text.strip()
        match = re.search(r"(S\d+)\.(\s*E\d+)\s*∙\s*(.*)", titre_brut)
        if match:
            saison = match.group(1).strip()
            numero = match.group(2).strip()
            titre = match.group(3).strip()
        else:
            saison, numero, titre = f"S{season_num}", "N/A", titre_brut

        description = a.select_one("div.ipc-html-content-inner-div").text.strip()
        date = a.select_one("div.sc-5372d523-5.jBUStp span").text.strip()
        note = a.select_one("span.ipc-rating-star--rating").text.strip()
        nbre_votants = a.select_one("span.ipc-rating-star--voteCount").text.strip()

        all_episodes.append({
            "titre": titre,
            "saison": saison,
            "épisode": numero,
            "résumé": description,
            "date de diffusion": date,
            "note": float(note.replace(",", ".")),
            "nombre de votants": nbre_votants
        })
#print(all_episodes)
df_all_seasons = pd.DataFrame(all_episodes)
df_all_seasons
#Deduplication
#df_all_seasons.drop_duplicates(subset=["titre", "saison", "épisode"], inplace=True)

# Présenter les 10 épisodes les mieux notés
#print(df_all_seasons.head(10))

,titre,saison,épisode,résumé,date de diffusion,note,nombre de votants
0,The One Where Monica Gets a Roommate,S1,E1,"Monica and the gang introduce Rachel to the ""r...","Sun, Dec 31, 2017",8.1,(11K)
1,The One with the Sonogram at the End,S1,E2,Ross finds out his ex-wife is pregnant. Rachel...,"Thu, Sep 29, 1994",7.8,(8.6K)
2,The One with the Thumb,S1,E3,Monica becomes irritated when everyone likes h...,"Thu, Oct 6, 1994",8.0,(8.1K)
3,The One with George Stephanopoulos,S1,E4,Joey and Chandler take Ross to a hockey game t...,"Thu, Oct 13, 1994",7.9,(7.9K)
4,The One with the East German Laundry Detergent,S1,E5,"Eager to spend time with Rachel, Ross pretends...","Thu, Oct 20, 1994",8.3,(7.9K)
...,...,...,...,...,...,...,...
230,The One with Princess Consuela,S10,E14,Phoebe ponders whether or not to changer her n...,"Thu, Feb 26, 2004",8.4,(5.1K)
231,The One Where Estelle Dies,S10,E15,Ross tries to get Rachel to go back to Ralph L...,"Thu, Apr 22, 2004",8.4,(4.8K)
232,The One with Rachel's Going Away Party,S10,E16,"The gang throws Rachel a goodbye party, during...","Thu, Apr 29, 2004",8.8,(5.5K)
233,The Last One: Part 1,S10,E17,Phoebe races Ross to the airport in a bid to s...,"Thu, May 6, 2004",9.6,(17K)


In [116]:
#Creation d une liste des épisodes avec itération d une boucle , recupération des titres avec une regular expression,
episodes = []
for a in soup.select("article.episode-item-wrapper"):

    titre_brut = a.select_one("div.ipc-title__text.ipc-title__text--reduced").text.strip()
    match = re.search(r"(S\d+)\.(\s*E\d+)\s*∙\s*(.*)", titre_brut)
    saison = match.group(1).strip()
    numero = match.group(2).strip()
    titre = match.group(3).strip()

    description = a.select_one("div.ipc-html-content-inner-div").text.strip()

    date = a.select_one("div.sc-5372d523-5.jBUStp span").text.strip()

    note = a.select_one("span.ipc-rating-star--rating").text.strip()

    nbre_votants = a.select_one("span.ipc-rating-star--voteCount").text.strip()
#génération d'un dataframe
    episodes.append({'titre': titre, "saison": saison, "épisode": numero, "résumé": description, "date de diffusion": date, 'note': note, "nombre de votants": nbre_votants})
df_saison1 = pd.DataFrame(episodes)
df_saison1.head(10)
# Tri par note décroissante
df_saison1_sorted = df_saison1.sort_values(by="note", ascending=False)

# Affichage des 10 meilleurs épisodes
print(df_saison1_sorted.head(10))

                                     titre saison épisode  \
16                    The Last One: Part 1    S10     E17   
17                    The Last One: Part 2    S10     E18   
10        The One Where the Stripper Cries    S10     E11   
15  The One with Rachel's Going Away Party    S10     E16   
7       The One with the Late Thanksgiving    S10      E8   
11           The One with Phoebe's Wedding    S10     E12   
1               The One Where Ross Is Fine    S10      E2   
2                  The One with Ross's Tan    S10      E3   
14              The One Where Estelle Dies    S10     E15   
13          The One with Princess Consuela    S10     E14   

                                               résumé  date de diffusion note  \
16  Phoebe races Ross to the airport in a bid to s...   Thu, May 6, 2004  9.6   
17  The six friends prepare to say goodbye and emb...   Thu, May 6, 2004  9.5   
10  Old secrets come to light when Ross and Chandl...   Thu, Feb 5, 2004  8.9   
15  

In [121]:
#Sauvegarde au format cesv du dataframe
df_all_seasons.to_csv("friends_episodes.csv", index=False, encoding="utf-8")
with open("friends_episodes.csv", "r", encoding="utf-8") as f:
    print(f.read().splitlines()[:5])


['titre,saison,épisode,résumé,date de diffusion,note,nombre de votants', 'The One Where Monica Gets a Roommate,S1,E1,"Monica and the gang introduce Rachel to the ""real world"" after she leaves her fiancé at the altar.","Sun, Dec 31, 2017",8.1,(11K)', 'The One with the Sonogram at the End,S1,E2,Ross finds out his ex-wife is pregnant. Rachel returns her engagement ring to Barry. Monica becomes stressed when her and Ross\'s parents come to visit.,"Thu, Sep 29, 1994",7.8,(8.6K)', 'The One with the Thumb,S1,E3,Monica becomes irritated when everyone likes her new boyfriend more than she does. Chandler resumes his smoking habit. Phoebe is given $7000 when she finds a thumb in a can of soda.,"Thu, Oct 6, 1994",8.0,(8.1K)', 'The One with George Stephanopoulos,S1,E4,"Joey and Chandler take Ross to a hockey game to take his mind off the anniversary of the first time he slept with Carol, while the girls become depressed when they realize they don\'t have a \'plan\'.","Thu, Oct 13, 1994",7.9,(7.9K

## Ressources utiles



https://www.crummy.com/software/BeautifulSoup/bs4/doc/

https://www.crummy.com/software/BeautifulSoup/bs4/doc/#css-selectors


